# Notebook configuration

In [68]:
# Configuration
shapefile_dir_path = '/home/bertrand/Documents/Projet Indus/WazeTT/data/input/bd_topo/'
shapefile_name = 'ZONE_VEGETATION.shp'
shapefile_path = shapefile_dir_path + shapefile_name

tiles_dir_path = '../data/input/bd_alti/'
tiles_name = 'dalles.shp'
tiles_path = tiles_dir_path + tiles_name

ascii_file_dir_path = '../data/input/bd_alti/'
ascii_file_name = 'BDALTIV2_25M_FXX_0825_6550_MNT_LAMB93_IGN69.asc'
ascii_file_path = ascii_file_dir_path + ascii_file_name

# Load the altitudes ascii file into an AltiData object

In [69]:
from alti_data import AltiData
alti_data = AltiData(ascii_file_path)

# Deduce its Bounding Box coordinates in Lamb93 (xll, yll)

In [70]:
alti_data_extreme_bb_coordinates_lamb93 = alti_data.calc_bb_extreme_coordinates(format='lamb93')
print(alti_data_extreme_bb_coordinates_lamb93)

alti_data_bb_angle_coordinates_lamb93 = alti_data.calc_bb_angle_coordinates(format='lamb93')
print(alti_data_bb_angle_coordinates_lamb93)

{'xmin': 824987.5, 'xmax': 849987.5, 'ymin': 6500012.5, 'ymax': 6525012.5}
{'nw': (824987.5, 6525012.5), 'ne': (849987.5, 6525012.5), 'se': (849987.5, 6500012.5), 'sw': (824987.5, 6500012.5)}


# Create ASCII table Bounding Box polygon in Lamb93

In [71]:
ascii_table_bb_polygon_lamb93 = alti_data.calc_bb_polygon(format='lamb93')
print(ascii_table_bb_polygon_lamb93)

POLYGON ((824987.5 6525012.5, 849987.5 6525012.5, 849987.5 6500012.5, 824987.5 6500012.5, 824987.5 6525012.5))


# Display a Polygon on world map

In [72]:
import geopandas as gpd
from pyproj import CRS, Transformer
import folium
import json
import shapely

In [73]:
alti_data_bb_polygon = json.dumps(shapely.geometry.mapping(alti_data.calc_bb_polygon(format='wgs84')),indent=4)

m = folium.Map(
    location=[45.6, 4.9],
    tiles="cartodbpositron",
    zoom_start=10,
)

folium.GeoJson(alti_data_bb_polygon, name="Bounding Box").add_to(m)

# Add the possibility to the user to display the layers or not (The layers are displayed by default.)
folium.LayerControl().add_to(m)

m

# Load the field shapefile

In [77]:
# Conversion of a pair of files (.shp + .shx) into a list of POLYGON included into the ASCII table Bounding Box
import geopandas as gpd

bbox_extreme_coordinates = (
                            alti_data_extreme_bb_coordinates_lamb93['xmin'],
                            alti_data_extreme_bb_coordinates_lamb93['xmax'],
                            alti_data_extreme_bb_coordinates_lamb93['ymin'],
                            alti_data_extreme_bb_coordinates_lamb93['ymax'],
)
gdf = gpd.read_file(
    shapefile_path,
    bbox=bbox_extreme_coordinates,
)

shapefile_polygones_lamb93 = gdf['geometry']
print(shapefile_polygones_lamb93)

0       POLYGON ((907748.000 6520508.100, 907742.900 6...
1       POLYGON ((910072.900 6523975.600, 910068.900 6...
2       POLYGON ((896896.000 6524342.400, 896913.800 6...
3       POLYGON ((896004.200 6520785.000, 895999.000 6...
4       POLYGON ((912960.900 6516887.800, 912958.500 6...
                              ...                        
9155    POLYGON ((913716.100 6520221.600, 913726.400 6...
9156    POLYGON ((913955.300 6520072.300, 913957.700 6...
9157    POLYGON ((910987.700 6520097.700, 911065.400 6...
9158    POLYGON ((915525.700 6520592.100, 915586.500 6...
9159    POLYGON ((915283.600 6523628.200, 915286.400 6...
Name: geometry, Length: 9160, dtype: geometry


# Plot the first polygon of the shapefile

In [ ]:
from coordinates_utils import wgs84_epsg_code

gdf_wgs84 = gdf.to_crs("EPSG:"+str(wgs84_epsg_code))
shapefile_polygones_wgs84 = gdf_wgs84['geometry']

m = folium.Map(
    location=[45.6, 4.9],
    tiles="cartodbpositron",
    zoom_start=10,
)

folium.GeoJson(shapefile_polygones_wgs84[0:10], name="Bounding Box").add_to(m)

# Add the possibility to the user to display the layers or not (The layers are displayed by default.)
folium.LayerControl().add_to(m)

m

# Creation of the field table to project the shapefiles onto

In [79]:
# Initializes the empty field 2D table
field_table = [['default_field' for col in range(alti_data.ncols)] for row in range(alti_data.nrows)]
print('Number of rows : {}'.format(len(field_table[0])))
print('Number of columns : {}'.format(len(field_table)))
print('Cells content : {}'.format(field_table[0][0]))

Number of rows : 1000
Number of columns : 1000
Cells content : default_field


# Calculate polygons bounding boxes

In [87]:
from coordinates_utils import bb_bounds_to_coord
polygon_bb_bounds = []
polygons_bb_coordinates = []
i=0
for polygon in shapefile_polygones_lamb93[0:10]:
    polygon_bb_bounds.append(polygon.bounds)
    polygons_bb_coordinates.append(bb_bounds_to_coord(polygon_bb_bounds[i]))
    i += 1
print(polygons_bb_coordinates)

[{'nw': (907737.6, 6520518.3), 'ne': (907748.2, 6520518.3), 'se': (907748.2, 6520504.1), 'sw': (907737.6, 6520504.1)}, {'nw': (910058.3, 6523980.8), 'ne': (910072.9, 6523980.8), 'se': (910072.9, 6523969.8), 'sw': (910058.3, 6523969.8)}, {'nw': (894041.9, 6524369.3), 'ne': (897677.2, 6524369.3), 'se': (897677.2, 6520992.0), 'sw': (894041.9, 6520992.0)}, {'nw': (895994.9, 6520796.3), 'ne': (896007.0, 6520796.3), 'se': (896007.0, 6520783.4), 'sw': (895994.9, 6520783.4)}, {'nw': (912946.4, 6517049.5), 'ne': (913001.9, 6517049.5), 'se': (913001.9, 6516812.2), 'sw': (912946.4, 6516812.2)}, {'nw': (908799.0, 6520412.9), 'ne': (908830.4, 6520412.9), 'se': (908830.4, 6520400.8), 'sw': (908799.0, 6520400.8)}, {'nw': (910151.7, 6520809.4), 'ne': (910196.1, 6520809.4), 'se': (910196.1, 6520786.2), 'sw': (910151.7, 6520786.2)}, {'nw': (904160.6, 6521720.8), 'ne': (904170.1, 6521720.8), 'se': (904170.1, 6521706.2), 'sw': (904160.6, 6521706.2)}, {'nw': (904647.7, 6521725.2), 'ne': (904661.1, 6521725.

# Calculate indexes of polygons BB relatively to the 2D altitudes table

In [99]:
from math import ceil

polygons = []
polygons_extreme_indexes = []
for polygon_bb_coordinates in polygons_bb_coordinates[0:10]:
    Xll_min = polygon_bb_coordinates["nw"][0]
    Xll_max = polygon_bb_coordinates["ne"][0]
    Yll_min = polygon_bb_coordinates["sw"][1]
    Yll_max = polygon_bb_coordinates["ne"][1]

    polygon_bb_extreme_indexes_into_table = {}
    polygon_bb_extreme_indexes_into_table['j_min'] = ceil(abs(alti_data.xllcorner_lamb93-Xll_min)/alti_data.cellsize) - 1
    polygon_bb_extreme_indexes_into_table['j_max'] = ceil(abs(alti_data.xllcorner_lamb93-Xll_max)/alti_data.cellsize) - 1
    polygon_bb_extreme_indexes_into_table['i_min'] = ceil(abs(alti_data.yllcorner_lamb93-Yll_max)/alti_data.cellsize) - 1
    polygon_bb_extreme_indexes_into_table['i_max'] = ceil(abs(alti_data.yllcorner_lamb93-Yll_min)/alti_data.cellsize) - 1
    polygons_extreme_indexes.append(polygon_bb_extreme_indexes_into_table)
    
    bb_angles_indexes_dict = (
                    polygon_bb_extreme_indexes_into_table['j_min'],
                    polygon_bb_extreme_indexes_into_table['i_min'],
                    polygon_bb_extreme_indexes_into_table['j_max'],
                    polygon_bb_extreme_indexes_into_table['i_max']
                )

    polygons.append(bb_bounds_to_coord(bb_angles_indexes_dict))

print(polygons)

[{'nw': (3310, 180), 'ne': (3310, 180), 'se': (3310, 179), 'sw': (3310, 179)}, {'nw': (3402, 41), 'ne': (3403, 41), 'se': (3403, 41), 'sw': (3402, 41)}, {'nw': (2762, 160), 'ne': (2907, 160), 'se': (2907, 25), 'sw': (2762, 25)}, {'nw': (2840, 169), 'ne': (2840, 169), 'se': (2840, 168), 'sw': (2840, 168)}, {'nw': (3518, 328), 'ne': (3520, 328), 'se': (3520, 318), 'sw': (3518, 318)}, {'nw': (3352, 184), 'ne': (3353, 184), 'se': (3353, 183), 'sw': (3352, 183)}, {'nw': (3406, 169), 'ne': (3408, 169), 'se': (3408, 168), 'sw': (3406, 168)}, {'nw': (3166, 132), 'ne': (3167, 132), 'se': (3167, 131), 'sw': (3166, 131)}, {'nw': (3186, 132), 'ne': (3186, 132), 'se': (3186, 131), 'sw': (3186, 131)}, {'nw': (3348, 161), 'ne': (3349, 161), 'se': (3349, 161), 'sw': (3348, 161)}]


# Creation of the cells center coordinates table

In [100]:
# Calculate cells center coordinates table
cells_center_coordinates = [[None for col in range(alti_data.ncols)] for row in range(alti_data.nrows)]
nw_coord_ref_lamb93 = (
                        alti_data.xllcorner_lamb93 + alti_data.cellsize/2,
                        alti_data.yllcorner_lamb93 - alti_data.cellsize/2
                    )

for row in range(len(cells_center_coordinates)):
    for col in range(len(cells_center_coordinates[0])):

        cell_center_coord = (
            nw_coord_ref_lamb93[0]+(col)*alti_data.cellsize,
            nw_coord_ref_lamb93[1]-(row)*alti_data.cellsize
        )
        cells_center_coordinates[row][col] = cell_center_coord

# Vectorize polygons

In [101]:
polygons_as_vectors = []
for polygon in polygons:
    vector = [
                polygon['nw'],
                polygon['ne'],
                polygon['se'],
                polygon['sw'],
            ]
    polygons_as_vectors.append(vector)
polygons_as_vectors

[[(3310, 180), (3310, 180), (3310, 179), (3310, 179)],
 [(3402, 41), (3403, 41), (3403, 41), (3402, 41)],
 [(2762, 160), (2907, 160), (2907, 25), (2762, 25)],
 [(2840, 169), (2840, 169), (2840, 168), (2840, 168)],
 [(3518, 328), (3520, 328), (3520, 318), (3518, 318)],
 [(3352, 184), (3353, 184), (3353, 183), (3352, 183)],
 [(3406, 169), (3408, 169), (3408, 168), (3406, 168)],
 [(3166, 132), (3167, 132), (3167, 131), (3166, 131)],
 [(3186, 132), (3186, 132), (3186, 131), (3186, 131)],
 [(3348, 161), (3349, 161), (3349, 161), (3348, 161)]]

# Fill the polygons into the field map

In [103]:
from winding_point_utils import wn_PnPoly

for p in range(0, len(polygons_as_vectors)):
    extreme_indexes = polygons_extreme_indexes[p]
    V = polygons_as_vectors[p]
    for i in range(extreme_indexes['i_min'],extreme_indexes['i_max']): # x = j = rows
        for j in range(extreme_indexes['j_min'],extreme_indexes['j_max']): # y = i = columns
            if wn_PnPoly(cells_center_coordinates[i][j],V):# if true, the pixel is inside the shape
                field_table[i][j] = 'wood'

IndexError: list index out of range

# Plot the 10 first polygons (blue) and their BB (red)?

# Merge all the tiles information into a single data structure (Jérémy)

In [ ]:
merge_all = []
Dalle1 = {}
Dalle1['alti']=ascii_table
Dalle1['milieu1']=[]
Dalle1['milieu2']=[]

merge_all.append(Dalle1)

# (Optionnal) Get shapefile BoundingBox coordinates
WGS84 coordinates system (lat, long)

In [36]:
# xml_path = gmd:MD_Metadata/gmd:identificationInfo/gmd:MD_DataIdentification/gmd:extent/gmd:EX_Extent/gmd:geographicElement/gmd:EX_GeographicBoundingBox
import xml.etree.ElementTree as ET
metadata_file_name = 'metadonnees.xml'
tree = ET.parse(shapefile_dir_path + metadata_file_name)
root = tree.getroot()

namespaces = {
                'gmd': '{http://www.isotc211.org/2005/gmd}',
                'gco': '{http://www.isotc211.org/2005/gco}'
            } # add more as needed

EX_GeographicBoundingBox_element = list(root.iter(namespaces['gmd']+'EX_GeographicBoundingBox'))[0]

west_bound_longitude_element = list(EX_GeographicBoundingBox_element.iter(namespaces['gmd']+'westBoundLongitude'))[0]
west_bound_longitude_decimal_element = list(west_bound_longitude_element.iter(namespaces['gco']+'Decimal'))[0]
west_bound_longitude_decimal_value = list(west_bound_longitude_decimal_element.itertext())[0]
print('west_bound_longitude_decimal_value : {}'.format(west_bound_longitude_decimal_value))

east_bound_longitude_element = list(EX_GeographicBoundingBox_element.iter(namespaces['gmd']+'eastBoundLongitude'))[0]
east_bound_longitude_decimal_element = list(east_bound_longitude_element.iter(namespaces['gco']+'Decimal'))[0]
east_bound_longitude_decimal_value = list(east_bound_longitude_decimal_element.itertext())[0]
print('east_bound_longitude_decimal_value : {}'.format(east_bound_longitude_decimal_value))

south_bound_latitude_element = list(EX_GeographicBoundingBox_element.iter(namespaces['gmd']+'southBoundLatitude'))[0]
south_bound_latitude_decimal_element = list(south_bound_latitude_element.iter(namespaces['gco']+'Decimal'))[0]
south_bound_latitude_decimal_value = list(south_bound_latitude_decimal_element.itertext())[0]
print('south_bound_latitude_decimal_value : {}'.format(south_bound_latitude_decimal_value))

north_bound_latitude_element = list(EX_GeographicBoundingBox_element.iter(namespaces['gmd']+'northBoundLatitude'))[0]
north_bound_latitude_decimal_element = list(north_bound_latitude_element.iter(namespaces['gco']+'Decimal'))[0]
north_bound_latitude_decimal_value = list(north_bound_latitude_decimal_element.itertext())[0]
print('north_bound_latitude_decimal_value : {}'.format(north_bound_latitude_decimal_value))

west_bound_longitude_decimal_value : 2.0629
east_bound_longitude_decimal_value : 7.1856
south_bound_latitude_decimal_value : 44.1155
north_bound_latitude_decimal_value : 46.8043
